In [ ]:
import torch.nn as nn
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from omegaconf import DictConfig
from loguru import logger

class MirrorUNet(nn.Module):
    def __init__(self):
        super().__init__()
        norm = "instance"
        self.pathology_model = UNet(
            spatial_dims=3,
            in_channels=2,
            out_channels=54,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            num_res_units=2,
            norm=norm,
        )
        self.anatomy_model = UNet(
            spatial_dims=3,
            in_channels=1, # ct
            out_channels=145,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            # num_res_units=num_res_units,
            norm="instance",
        )
            
m = MirrorUNet()
print(m.pathology_model.model)

Sequential(
  (0): ResidualUnit(
    (conv): Sequential(
      (unit0): Convolution(
        (conv): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (adn): ADN(
          (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
      (unit1): Convolution(
        (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (adn): ADN(
          (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
    )
    (residual): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  )
  (1): SkipConnection(
    (submodule): Sequential(
      (0): ResidualUnit(
        (conv): Sequential(
          (unit0): Convolution(
            (conv): Conv3

In [3]:
from monai.networks.layers import Norm, SkipConnection
print(m.pathology_model.model[1].submodule[-1][-1].conv.unit0.conv)

Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))


In [8]:
import torch
input_data = torch.randn(1, 2, 96, 96, 96)  # B, C, H, W, D

print(input_data.shape[2:])

# def hook_fn(module, input, output):
#     print(f"output shape in bottleneck: {output.shape}")

# # 1, 256, 6, 6, 6

# from monai.networks.layers import Norm, SkipConnection
# for i, layer in enumerate(m.pathology_model.model):
#     if i == 1 and isinstance(layer, SkipConnection):
#         conv_256 = layer.submodule[1].submodule[1].submodule[0].conv.unit0.conv
#         conv_512 = layer.submodule[1].submodule[1].submodule[1].submodule[0].conv.unit0.conv
#         print(conv_256)
#         print(conv_512)
# bottleneck = conv_512 # last layer of the down_layers
# hook = bottleneck.register_forward_hook(hook_fn)


# with torch.no_grad():
#     output = m.pathology_model(input_data)
    

# hook.remove()


# print(f"input_data shape: {input_data.shape}")
# print(f"output shape: {output.shape}")

torch.Size([96, 96, 96])


In [21]:
def get_last_conv(strides, offset=0):
    """
    Dynamically construct the path to the last convolution layer
    and evaluate it to access the target Conv3d layer.

    Args:
        layer: The input SkipConnection layer.
        strides: The number of strides (depth of traversal).
    Returns:
        The last Conv3d layer.
    """
    #submodule[1] -> submodule[1] -> ...
    path = "layer"
    for _ in range(strides - 1 - offset):
        path += ".submodule[1]"
    path += ".submodule.conv.unit0.conv"  
    
    result = eval(path)
    print(path)
    return result
from monai.networks.layers import Norm, SkipConnection
for i, layer in enumerate(m.anatomy_model.model):
    # print(f"layer {i}: {layer}")
    if i == 1:
        if isinstance(layer, SkipConnection):
            last_conv_512 = layer.submodule[1].submodule[1].submodule[1].submodule[0].conv
            
            print(last_conv_512)
            # last_conv = get_last_conv(5)
            # print(last_conv)

Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))


In [4]:
import torch.nn as nn
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from omegaconf import DictConfig
from loguru import logger

class MirrorUNet(nn.Module):
    def __init__(self):
        super().__init__()
        norm = "instance"
        self.pathology_model = UNet(
            spatial_dims=3,
            in_channels=2,
            out_channels=54,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            num_res_units=2,
            norm=norm,
        )
        self.anatomy_model = UNet(
            spatial_dims=3,
            in_channels=1, # ct
            out_channels=145,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            # num_res_units=num_res_units,
            norm="instance",
        )
            
m = MirrorUNet()
print(m.anatomy_model.model)

Sequential(
  (0): Convolution(
    (conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (adn): ADN(
      (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (D): Dropout(p=0.0, inplace=False)
      (A): PReLU(num_parameters=1)
    )
  )
  (1): SkipConnection(
    (submodule): Sequential(
      (0): Convolution(
        (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (adn): ADN(
          (N): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
      (1): SkipConnection(
        (submodule): Sequential(
          (0): Convolution(
            (conv): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
            (adn): ADN(
              (N): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_run

In [7]:
print(m.anatomy_model.model[1].submodule[-1].conv)

ConvTranspose3d(128, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1))
